In [1]:
!pip install neo4j

Looking in indexes: https://pkg.pjlab.org.cn/repository/pypi-proxy/simple/
  Using cached https://pkg.pjlab.org.cn/repository/pypi-proxy/packages/neo4j/5.20.0/neo4j-5.20.0.tar.gz (202 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=3887ac479c5868f81e3d2327cdfd15dea7f62f0fd6698689955e13aff1d524e3
  Stored in directory: /mnt/petrelfs/songmingyang/.cache/pip/wheels/b4/47/40/daad4d061c4ed65b16eace33258a0cb143308e971cc2444d1c
Successfully built neo4j


In [6]:
import os
os.environ['PATH']

'/mnt/petrelfs/share/gcc/gcc-7.5.0/bin:/mnt/petrelfs/songmingyang/softwares/install/cmake/bin:/mnt/petrelfs/share/gcc/gcc-7.5.0/bin:/mnt/petrelfs/share/cuda-12.1/bin:/mnt/petrelfs/songmingyang/.vscode-server/bin/8b3775030ed1a69b13e4f4c628c612102e30a681/bin/remote-cli:/mnt/petrelfs/songmingyang/softwares/install/cmake/bin:/mnt/petrelfs/share/gcc/gcc-7.5.0/bin:/mnt/petrelfs/share/cuda-12.1/bin:/mnt/petrelfs/songmingyang/softwares/install/cmake/bin:/mnt/petrelfs/share/gcc/gcc-7.5.0/bin:/mnt/petrelfs/share/cuda-12.1/bin:/mnt/petrelfs/songmingyang/anaconda3/envs/vllm/bin:/mnt/petrelfs/songmingyang/anaconda3/condabin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/mnt/petrelfs/songmingyang/softwares/install/postgres/bin:/mnt/petrelfs/songmingyang/softwares/neo4j-enterprise-5.19.0/bin:/mnt/petrelfs/songmingyang/.local/bin:/mnt/petrelfs/songmingyang/bin:/mnt/petrelfs/songmingyang/softwares/install/postgres/bin:/mnt/petrelfs/songmingyang/softwares/neo4j-enterprise-5.19.0/bin:/mnt/petrelfs/so

In [2]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection 

model_id = "/mnt/petrelfs/songmingyang/songmingyang/model/mm/grounding-dino-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

image_url = "/mnt/petrelfs/songmingyang/songmingyang/data/mm/imgs/coco/train2017/000000334872.jpg"
# image = Image.open(requests.get(image_url, stream=True).raw)
image = Image.open(image_url)
# Check for cats and remote controls
# VERY important: text queries need to be lowercased + end with a dot
text = "person.man.tree.sled."

inputs = processor(images=image, text=text, return_tensors="pt").to(device)


In [6]:
inputs.keys()
for k,v in inputs.items():
    print(k, v.shape, v.dtype)

input_ids torch.Size([1, 11]) torch.int64
token_type_ids torch.Size([1, 11]) torch.int64
attention_mask torch.Size([1, 11]) torch.int64
pixel_values torch.Size([1, 3, 800, 1199]) torch.float32
pixel_mask torch.Size([1, 800, 1199]) torch.int64


In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

results = processor.post_process_grounded_object_detection(
    outputs,
    inputs.input_ids,
    box_threshold=0.4,
    text_threshold=0.3,
    target_sizes=[image.size[::-1]]
)


In [3]:
results

[{'scores': tensor([0.5016, 0.4897, 0.4285], device='cuda:0'),
  'labels': ['person man', 'person man', 'tree'],
  'boxes': tensor([[4.7177e+02, 6.9915e+01, 6.1644e+02, 3.9339e+02],
          [3.1403e+02, 1.0249e+02, 3.9680e+02, 2.6103e+02],
          [1.0874e-01, 2.1556e-01, 6.3997e+02, 2.2572e+02]], device='cuda:0')}]

## Parallel re-coding

In [10]:
results

[{'scores': tensor([0.5235, 0.4809, 0.4233], device='cuda:0'),
  'labels': ['person man', 'person man', 'tree'],
  'boxes': tensor([[4.7213e+02, 6.9993e+01, 6.1626e+02, 3.9340e+02],
          [3.1390e+02, 1.0239e+02, 3.9687e+02, 2.6094e+02],
          [8.2645e-02, 1.8268e-01, 6.3993e+02, 2.2573e+02]], device='cuda:0')},
 {'scores': tensor([0.7973], device='cuda:0'),
  'labels': ['cat'],
  'boxes': tensor([[403.1876, 194.3045, 446.1494, 254.9202]], device='cuda:0')}]

In [16]:
processor.tokenizer.pad_token_id

0

In [14]:
inputs = processor(images=[image,image2], text=[text,text2], return_tensors="pt").to(device)
processor(text=text)

{'input_ids': [101, 2711, 1012, 2158, 1012, 3392, 1012, 22889, 2098, 1012, 4937, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
from accelerate import PartialState

In [2]:
# from py2neo import *
from neo4j import GraphDatabase
uri = "neo4j://10.140.54.61:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))
gds_version_query = "call dbms.components() yield name, versions, edition unwind versions as version return name, version, edition;"
# gds_version_query = "match (n) return count(n)"
# Create a driver session
with driver.session() as session:
    results = session.run(gds_version_query).data()
    print ("Neo4j Kernel version :" + str(results[0]['version']))


Neo4j Kernel version :5.19.0


In [12]:
query = "match (n) return n.lemma;"
# query = "match (a)-[r:CO_OCCURRENCE]->(b) return a.lemma, r.count, b.lemma;"
with driver.session(database="lcs558kllama") as session:
    object_results = session.run(query).data()

In [13]:
objects = [i["n.lemma"] for i in object_results]
len(objects)

95742

In [20]:
def get_n_items(word_list,begin=0,length=10000):
    res = ""
    for i in range(begin,min(begin+length,len(word_list))):
        res += word_list[i]+"."
    return res

res = get_n_items(objects)
        

In [21]:
res

'ellen hench.coffee table.clothes.snowmobile.cushion.Beth Hayn.hedgehog.wine expert.beer bottle.park.coconut wax.slider.manisha royaz.sunflowers.park bench.app.groom.ignition switch.postage stamp.music festival.kate thompson.skulls.dragon.blue car.hooded sweatshirt.businesses.diner.button.clematis.bride.coconut oil.driving lights.snow.android.stone floor.apron.mandir.time calculator.slice.drag strip.picture.eyeglasses.jillian guthrie.cloth.Voyager.villages.harold.face masks.land.prom dress.machine gun.classroom.apple logo.email.tart.committee.shops.truck.water hose.ashton kutcher.conference room.spinach.stuntizer.counting bears.airline workers.kindles.comics.batman.cookie.oatmeal.parliament.boot.Steve Jobs.application.yacht.speakers.cubes.rams.poppy.planter.women.harbour.metal.wire.mtv awards.superman.soldier.wheel.glo.steering.helmet.tent.airport.members.temple.buffet.belt.jewelry box.frittes.bullet shell.puppy.bodysuit.internet map.guy pearce.tires.gun.workers.norton antivirus.shelvi

In [22]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection 

model_id = "/mnt/petrelfs/songmingyang/songmingyang/model/mm/grounding-dino-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

image_url = "/mnt/petrelfs/songmingyang/songmingyang/data/mm/imgs/coco/train2017/000000334872.jpg"
image2_url = "/mnt/petrelfs/songmingyang/songmingyang/data/mm/imgs/coco/train2017/000000052846.jpg"
# image = Image.open(requests.get(image_url, stream=True).raw)
image = Image.open(image_url)
image2 = Image.open(image2_url)
# Check for cats and remote controls
# VERY important: text queries need to be lowercased + end with a dot
text = "person.man.tree.sled.cat."
text2 = "person.man.tree.sled.cat."

inputs = processor(images=[image,image2], text=[res,res], return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)

results = processor.post_process_grounded_object_detection(
    outputs,
    inputs["input_ids"],
    box_threshold=0.4,
    text_threshold=0.3,
    target_sizes=[image.size[::-1],image2.size[::-1]]
)

Could not load the custom kernel for multi-scale deformable attention: /mnt/petrelfs/songmingyang/.cache/torch_extensions/py310_cu121/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /mnt/petrelfs/songmingyang/.cache/torch_extensions/py310_cu121/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /mnt/petrelfs/songmingyang/.cache/torch_extensions/py310_cu121/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /mnt/petrelfs/songmingyang/.cache/torch_extensions/py310_cu121/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No

RuntimeError: The size of tensor a (29384) must match the size of tensor b (256) at non-singleton dimension 2

In [8]:
results

'ellen hench.coffee table.clothes.snowmobile.cushion.Beth Hayn.hedgehog.wine expert.beer bottle.park.coconut wax.slider.manisha royaz.sunflowers.park bench.app.groom.ignition switch.postage stamp.music festival.kate thompson.skulls.dragon.blue car.hooded sweatshirt.businesses.diner.button.clematis.bride.coconut oil.driving lights.snow.android.stone floor.apron.mandir.time calculator.slice.drag strip.picture.eyeglasses.jillian guthrie.cloth.Voyager.villages.harold.face masks.land.prom dress.machine gun.classroom.apple logo.email.tart.committee.shops.truck.water hose.ashton kutcher.conference room.spinach.stuntizer.counting bears.airline workers.kindles.comics.batman.cookie.oatmeal.parliament.boot.Steve Jobs.application.yacht.speakers.cubes.rams.poppy.planter.women.harbour.metal.wire.mtv awards.superman.soldier.wheel.glo.steering.helmet.tent.airport.members.temple.buffet.belt.jewelry box.frittes.bullet shell.puppy.bodysuit.internet map.guy pearce.tires.gun.workers.norton antivirus.shelvi

In [16]:
len(res)

TypeError: object of type 'NoneType' has no len()